In [4]:
import pymongo
from pymongo import MongoClient

In [5]:
try:
    uri = "mongodb://localhost:27017/"
    client = MongoClient(uri)
    db = client["Ecommerce_NOSQL_mongodb10"]
except Exception as e:
    raise Exception(
        f"The following error occurred: {e}")

In [6]:
orders = db.create_collection("orders")
products = db.create_collection("products")
order_items = db.create_collection("order_items")
customers = db.create_collection("customers")

In [7]:
db.list_collection_names()

['orders', 'products', 'order_items', 'customers']

CREATING VALIDATORS

In [8]:
products_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "title": "'products' Object Validation",
        "required": ["products_name", "stock_quantity", "category", "price"],
        "properties": {
            "products_name": {
                "bsonType": "string",
                "description": "'products_name' must be a string and is required"
            },
            "stock_quantity": {
                "bsonType": "double",
                "description": "'stock_quantity' must be a double and is required"
            },
            "category": {
                "bsonType": "string",
                "description": "'category' must be a string and is required"
            },
            "price": {
                "bsonType": "int",
                "description": "'price' must be an integer and is required"
            }
        }
    }
}

In [9]:
customer_validator = {
            "$jsonSchema": {
                    "bsonType":"object",
                    "title": "'customers' Object Validation",
                    "required": [ "name", "email", "phone_number", "address" ],
                    "properties": {
                            "name": {
                                "bsonType": "string",
                                "description": "name' must be a string and is required"
                                    },
                            "email": {
                                "bsonType":"string",
                                "description": "'email' must be a string"
                                     },
                            "phone_number": {
                                 "bsonType":  "string",
                                 "description": "phone number must be a string"
                                            }
         }
      }
   }

In [10]:
order_items_validator = { 
            "$jsonSchema": {
                    "bsonType": "object",
                    "title": "'order_items' Object Validation",
                    "required": ["quantity", "price" ],
                    "properties": {
                            "quantity": {
                                "bsonType": "double",
                                "description": "name' must be a string and is required"
                                        },
                            "price": {
                                "bsonType":"int",
                                "description": "price must be an integar"
                                     }
         }
      }
   }

In [11]:
orders_validator = {
            "$jsonSchema": {
                    "bsonType": "object",
                    "title": "'orders' Object Validation",
                    "required": [ "order_date", "total_amount" ],
                    "properties": {
                            "order_date": {
                                "bsonType": "date",
                                "description": "must be a date and is required"
                                },
                            "total_amount": {
                                "bsonType":"double",
                                "description": "total amount can be double"
                                }
         }
      }
   }

VALIDATING THE COLLECTIONS

In [12]:
db.command({
    "collMod": "products",
    "validator": products_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [13]:
db.command({
    "collMod": "customers",
    "validator": customer_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [14]:
db.command({
    "collMod": "order_items",
    "validator": order_items_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

In [15]:
db.command({
    "collMod": "orders",
    "validator": orders_validator,
    "validationLevel": "strict"
})

{'ok': 1.0}

GENERATING FAKE DATA FOR THE COLLECTIONS AND INSERTING THEM INTO THE COLLECTIONS

In [16]:
from faker import Faker
import random


AltschoolFake = Faker()


# Generate random data
def generate_customers_data(rows):
    customers_data = []
    for i in range(rows):
        name = AltschoolFake.name()
        email = AltschoolFake.email()
        address = AltschoolFake.address()
        phone_number = AltschoolFake.numerify('##########')
        customers_data.append({ 
            "name": name,
            "email": email,
            "phone_number": phone_number,
            "address" : address
        })
    return customers_data

data_for_customers = generate_customers_data(1)
c = customers.insert_many(data_for_customers)
customer_ids = c.inserted_ids

In [17]:
categories = [
    "Electronics", "Home Appliances", "Furniture", "Clothing",
    "Sports", "Toys", "Books", "Beauty", "Automotive", "Groceries"
]
def generate_products_data(rows):
    products_data = []
    for i in range(rows):
        products_name = AltschoolFake.word()
        category = random.choice(categories)
        price = random.randint(1,100)
        stock_quantity = float(random.randint(1, 100))
        products_data.append({ 
            "products_name": products_name,
            "category": category,
            "price": price,
            "stock_quantity" : stock_quantity
        })
    return products_data


data_for_products = generate_products_data(1)
p = products.insert_many(data_for_products)
products_id = c.inserted_ids

In [18]:
def generate_orders_data(customer_ids, rows):
    orders_data = []
    for i in range(rows):
        customer_id = random.choice(customer_ids)
        order_date = AltschoolFake.date_time_this_year()
        total_amount = float(random.randint(100, 10000))
        orders_data.append({ 
            "customer_id" : customer_id,
            "order_date": order_date,
            "total_amount": total_amount,
        })
    return orders_data

customer_ids = [customer['_id'] for customer in db.customers.find()]
data_for_orders = generate_orders_data(customer_ids, 1) 
o = orders.insert_many(data_for_orders)
orders_id = o.inserted_ids

In [19]:
def generate_order_items_data(orders_id, products_id, rows):
    order_items_data = []
    for i in range(rows):
        order_id = random.choice(orders_id)
        product_id = random.choice(products_id)
        quantity = float(random.randint(0, 100))
        price = random.randint(1, 1000)
        order_items_data.append({
            "order_id" : order_id,
            "product_id" : product_id,
            "quantity" : quantity,
            "price": price
        })
    return order_items_data
    
orders_ids = [order['_id'] for order in db.orders.find()]
products_ids = [product['_id'] for product in db.products.find()]
data_for_order_items = generate_order_items_data(orders_ids, products_ids, 1)
order_items.insert_many(data_for_order_items)

InsertManyResult([ObjectId('676bb88015e594a62a05a154')], acknowledged=True)

In [20]:
#Fixing Delievery date issue on the orders table 
from datetime import datetime, timedelta 
import random


# Function to generate a random delivery date
def random_delivery_date(order_date):
    # Generate a random number of days between 1 and 30
    days = random.randint(1, 30)
    return order_date + timedelta(days=days)

# Update each document with a random delivery date
for order in orders.find():
    order_date = order['order_date']
    delivery_date = random_delivery_date(order_date)
    orders.update_one(
        {"_id": order["_id"]},
        {"$set": {"delivery_date": delivery_date}}
    )




TASK 4: ADVANCED FEAUTURES 


1.Implement Transactions:Simulate an order creation process that updates the orders and inventory collections atomically.

In [ ]:

from datetime import datetime


inventory = db.create_collection('inventory')  #create an inventory collection on the database since it doesnt exist

# Start a session
with client.start_session() as session:
    with session.start_transaction():
        try:
            # Insert a new order
            order = {
                "order_id": 5002,
                "customer_id": 2,
                "order_date": datetime.now(),
                "status": "Pending"
            }
            db.orders.insert_one(order, session=session)

            # Update inventory
            inventory_update = {
                "$inc": {"quantity": -1}
            }
            db.inventory.update_one({"product_id": 101}, inventory_update, session=session)

            # Commit the transaction
            session.commit_transaction()
            print("Transaction committed.")
        except Exception as e:
            # Abort the transaction in case of an error
            session.abort_transaction()
            print(f"Transaction aborted: {e}")


2. Change stream : Monitor real-time changes in the orders collection.

In [ ]:

# Watch the orders collection for changes
with db.orders.watch() as stream:
    for change in stream:
        print(f"Change detected: {change}")

# Watch the orders collection for changes
with db.orders.watch() as stream:
    for change in stream:
        print(f"Change detected: {change}")


3. Apply Schema Validation:Ensure all documents in the products collection include a valid price.

In [ ]:
order_items_validator = { 
            "$jsonSchema": {
                    "bsonType": "object",
                    "title": "'order_items' Object Validation",
                    "required": ["quantity", "price" ],
                    "properties": {
                            "quantity": {
                                "bsonType": "double",
                                "description": "name' must be a string and is required"
                                        },
                            "price": {
                                "bsonType":"int",
                                "minimum": 0,
                                "description": "price must be an integar"
                                     }
         }
      }
   }